In [1]:
from io import StringIO
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
from datetime import datetime

In [3]:
def add_log(message):
    with open('code_log.txt','a') as f:
        f.write(f"{datetime.now()}: {message}\n")

In [39]:
def extract(url,table_atrb):
    soup=BeautifulSoup(requests.get(url).text,'html.parser')
    table=soup.find('span',string=table_atrb).find_next('table')
    df=pd.read_html(StringIO(str(table)))[0]
    add_log("Data extraction completed! Now starting Transformation process")
    return df
    

In [131]:
def transform(df,csv_path):
    exchange_rate=pd.read_csv(csv_path,index_col=0).to_dict()['Rate']
    df['MC_GBP_Billion']=round(df['Market cap (US$ billion)']*exchange_rate['GBP'],2)
    df['MC_EUR_Billion']=round(df['Market cap (US$ billion)']*exchange_rate['EUR'],2)
    df['MC_INR_Billion']=round(df['Market cap (US$ billion)']*exchange_rate['INR'],2)
    # print(df)
    add_log("Data Transformation completed now moving to Loading process")
    return df

In [135]:
def load_to_csv(df,output_path):
    df.to_csv(output_path)
    add_log("Data Saved to cdv path :) ")



In [137]:
def load_to_db(df,sql_connection,table_name):
    df.to_sql(table_name,sql_connection,if_exists='replace',index =False)
    add_log("Data loaded to database as a table")


In [173]:
def run_query(query_statement,sql_connection):
    cursor=sql_connection.cursor()
    cursor.execute(query_statement)
    result=cursor.fetchall()
    add_log("Process completed")
    return result
    

In [175]:
if __name__ == "__main__":
    url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
    output_csv_path = './Largest_banks_data.csv'
    database_name = './Banks.db'
    table_name = 'largest_banks'

    add_log('Preliminaries complete. Initiating ETL process')

    try:
        df = extract(url, 'By market capitalization')
        df = transform(df, "./exchange_rate.csv")
        load_to_csv(df, output_csv_path)

        # Load to Database and run queries
        with sqlite3.connect(database_name) as conn:
            load_to_db(df, conn, table_name)
            print(run_query('SELECT * FROM largest_banks ', conn))
            print(run_query('SELECT AVG(MC_GBP_Billion) FROM largest_banks', conn))
            print(run_query('SELECT "Bank name" FROM largest_banks LIMIT 5', conn))

    except Exception as e:
        add_log(f"Error encountered: {e}")
        raise


[(1, 'JPMorgan Chase', 432.92, 346.34, 419.93, 36798.2), (2, 'Bank of America', 231.52, 185.22, 224.57, 19679.2), (3, 'Industrial and Commercial Bank of China', 194.56, 155.65, 188.72, 16537.6), (4, 'Agricultural Bank of China', 160.68, 128.54, 155.86, 13657.8), (5, 'HDFC Bank', 157.91, 126.33, 153.17, 13422.35), (6, 'Wells Fargo', 155.87, 124.7, 151.19, 13248.95), (7, 'HSBC Holdings PLC', 148.9, 119.12, 144.43, 12656.5), (8, 'Morgan Stanley', 140.83, 112.66, 136.61, 11970.55), (9, 'China Construction Bank', 139.82, 111.86, 135.63, 11884.7), (10, 'Bank of China', 136.81, 109.45, 132.71, 11628.85)]
[(151.987,)]
[('JPMorgan Chase',), ('Bank of America',), ('Industrial and Commercial Bank of China',), ('Agricultural Bank of China',), ('HDFC Bank',)]
